In [1]:
from google.colab import drive
drive.flush_and_unmount()  # 卸载之前的挂载


Drive not mounted, so nothing to flush and unmount.


##start

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.decomposition import NMF
import cv2
from PIL import Image
import random

In [2]:
reference_image_path = '/content/drive/MyDrive/target_image.tif'
reference_image = Image.open(reference_image_path)


##preprocess

###file structure

In [1]:
import os
from collections import defaultdict

# file type
def count_file_types(dataset_path):
    file_type_counts = defaultdict(lambda: defaultdict(int))

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            file_extension = os.path.splitext(file)[1].lower()
            folder_name = os.path.basename(root)
            file_type_counts[folder_name][file_extension] += 1

    return file_type_counts


def print_file_type_counts(file_type_counts):
    for folder, types in file_type_counts.items():
        print(f"\n folder '{folder}' file type:")
        for file_type, count in types.items():
            print(f"  {file_type}: {count} files")


dataset_path = '/content/drive/MyDrive/demo_dataset'


file_type_counts = count_file_types(dataset_path)
print_file_type_counts(file_type_counts)



 folder 'MUSC_STROMA' file type:
  .tif: 904 files
  .png: 116 files

 folder 'ADI' file type:
  .png: 199 files
  .tif: 821 files
  .tif（副本）: 1 files

 folder 'LYM' file type:
  .tif: 922 files
  .png: 78 files

 folder 'NORM' file type:
  .tif: 801 files
  .png: 199 files

 folder 'DEBRIS_MUCUS' file type:
  .tif: 876 files
  .png: 124 files

 folder 'TUM' file type:
  .png: 182 files
  .tif: 818 files


In [7]:
import os
from collections import defaultdict

# file type
def count_file_types(dataset_path):
    file_type_counts = defaultdict(lambda: defaultdict(int))

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            file_extension = os.path.splitext(file)[1].lower()
            folder_name = os.path.basename(root)
            file_type_counts[folder_name][file_extension] += 1

    return file_type_counts


def print_file_type_counts(file_type_counts):
    for folder, types in file_type_counts.items():
        print(f"\n folder '{folder}' file type:")
        for file_type, count in types.items():
            print(f"  {file_type}: {count} files")


dataset_path = '/content/drive/MyDrive/demo_dataset_preprocessed/macenko'


file_type_counts = count_file_types(dataset_path)
print_file_type_counts(file_type_counts)



 folder 'MUSC_STROMA' file type:
  .tif: 904 files
  .png: 116 files

 folder 'ADI' file type:
  .png: 199 files
  .tif: 821 files

 folder 'LYM' file type:
  .tif: 922 files
  .png: 78 files

 folder 'NORM' file type:
  .tif: 801 files
  .png: 199 files

 folder 'DEBRIS_MUCUS' file type:
  .tif: 876 files
  .png: 124 files

 folder 'TUM' file type:
  .png: 182 files
  .tif: 818 files


In [6]:
import os
import shutil


dataset_path = '/content/drive/MyDrive/demo_dataset'
preprocessed_path = '/content/drive/MyDrive/demo_dataset_preprocessed'


if not os.path.exists(preprocessed_path):
    os.makedirs(preprocessed_path)


macenko_path = os.path.join(preprocessed_path, 'macenko')
gaussian_path = os.path.join(preprocessed_path, 'gaussian')
rotation_path = os.path.join(preprocessed_path, 'rotation')

for path in [macenko_path, gaussian_path, rotation_path]:
    if not os.path.exists(path):
        os.makedirs(path)

In [7]:
import os


preprocessed_path = '/content/drive/MyDrive/demo_dataset_preprocessed'


paths = {
    'macenko': os.path.join(preprocessed_path, 'macenko'),
    'gaussian': os.path.join(preprocessed_path, 'gaussian'),
    'rotation': os.path.join(preprocessed_path, 'rotation'),
    'macenko_gaussian': os.path.join(preprocessed_path, 'macenko_gaussian'),
    'macenko_rotation': os.path.join(preprocessed_path, 'macenko_rotation'),
    'gaussian_rotation': os.path.join(preprocessed_path, 'gaussian_rotation'),
    'all_methods': os.path.join(preprocessed_path, 'all_methods'),
}


for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)


In [8]:
import os
print(os.listdir('/content/drive/MyDrive/'))


['NCT-CRC-HE-100K-NONORM.zip', '未命名绘图.drawio', 'Weekly Tutorial Presentation template (1).gslides', 'Client Meeting Minutes template.gdoc', 'Week8 pre.gslides', 'Client Meeting Minutes template (1).gdoc', 'demo_dataset', 'target_image.tif', 'demo_dataset_preprocessed', 'abl.ipynb']


In [8]:
import os
if os.path.exists('/content/drive/MyDrive/target_image.tif'):
    print("File exists!")
else:
    print("File not found.")

File exists!


In [7]:
# 列出文件路径而不仅仅是文件名
for file in os.listdir('/content/drive/MyDrive/'):
    print(file)


NCT-CRC-HE-100K-NONORM.zip
未命名绘图.drawio
Weekly Tutorial Presentation template (1).gslides
Client Meeting Minutes template.gdoc
Week8 pre.gslides
Client Meeting Minutes template (1).gdoc
demo_dataset
target_image.tif
demo_dataset_preprocessed
abl.ipynb


In [1]:
!ls /content/drive/


ls: cannot access '/content/drive/': No such file or directory


In [5]:
# Macenko stain normalization function
def apply_macenko(image, reference_image):
    # Convert input image and reference image to NumPy arrays
    image_np = np.array(image)
    ref_image_np = np.array(reference_image)

    # Reshape the image and reference image to 2D arrays (flattened)
    height, width, _ = image_np.shape
    image_flat = image_np.reshape((-1, 3))  # Flatten the image
    ref_flat = ref_image_np.reshape((-1, 3))  # Flatten the reference image

    # Non-negative Matrix Factorization (NMF)
    nmf = NMF(n_components=2)
    W_image = nmf.fit_transform(image_flat)  # Apply NMF to the input image
    H_image = nmf.components_

    W_ref = nmf.fit_transform(ref_flat)  # Apply NMF to the reference image
    H_ref = nmf.components_

    # Reconstruct the stain matrix using W_image and H_ref
    stains = np.dot(W_image, H_ref)
    stains = stains.reshape((height, width, 3))

    # Normalize the stain matrix to [0, 255] for image display
    stains = (stains - np.min(stains)) / (np.max(stains) - np.min(stains)) * 255

    return stains.astype(np.uint8)

###Macenko

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.decomposition import NMF

# Macenko stain normalization function
def apply_macenko(image, reference_image):
    # Convert input image and reference image to NumPy arrays
    image_np = np.array(image)
    ref_image_np = np.array(reference_image)

    # Reshape the image and reference image to 2D arrays (flattened)
    height, width, _ = image_np.shape
    image_flat = image_np.reshape((-1, 3))  # Flatten the image
    ref_flat = ref_image_np.reshape((-1, 3))  # Flatten the reference image

    # Non-negative Matrix Factorization (NMF)
    nmf = NMF(n_components=2)
    W_image = nmf.fit_transform(image_flat)  # Apply NMF to the input image
    H_image = nmf.components_

    W_ref = nmf.fit_transform(ref_flat)  # Apply NMF to the reference image
    H_ref = nmf.components_

    # Reconstruct the stain matrix using W_image and H_ref
    stains = np.dot(W_image, H_ref)
    stains = stains.reshape((height, width, 3))

    # Normalize the stain matrix to [0, 255] for image display
    stains = (stains - np.min(stains)) / (np.max(stains) - np.min(stains)) * 255

    return stains.astype(np.uint8)

# Dataset paths
dataset_path = '/content/drive/MyDrive/demo_dataset'  # Input dataset path
save_base_path = '/content/drive/MyDrive/demo_dataset_preprocessed/macenko'  # Output path for Macenko-processed images

# Load the reference image
reference_image_path = '/content/drive/MyDrive/target_image.tif'  # Replace with your target image path
reference_image = Image.open(reference_image_path)

# Process all images in the dataset and apply Macenko normalization
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('.tif', '.png')):  # Only process .tif and .png images
            file_path = os.path.join(root, file)
            image = Image.open(file_path)

            # Apply Macenko normalization
            processed_image = apply_macenko(image, reference_image)

            # Preserve original folder structure in the output folder
            relative_path = os.path.relpath(root, dataset_path)
            save_folder = os.path.join(save_base_path, relative_path)

            # Create the folder if it doesn't exist
            if not os.path.exists(save_folder):
                os.makedirs(save_folder)

            # Save the processed image in the original format
            save_extension = file.split('.')[-1].lower()
            save_path = os.path.join(save_folder, file)

            if save_extension == 'tif':
                Image.fromarray(processed_image).save(save_path, format='TIFF')
            elif save_extension == 'png':
                Image.fromarray(processed_image).save(save_path, format='PNG')

print("Macenko stain normalization completed for all images.")


In [8]:
import os
from collections import defaultdict

# file type
def count_file_types(dataset_path):
    file_type_counts = defaultdict(lambda: defaultdict(int))

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            file_extension = os.path.splitext(file)[1].lower()
            folder_name = os.path.basename(root)
            file_type_counts[folder_name][file_extension] += 1

    return file_type_counts


def print_file_type_counts(file_type_counts):
    for folder, types in file_type_counts.items():
        print(f"\n folder '{folder}' file type:")
        for file_type, count in types.items():
            print(f"  {file_type}: {count} files")


dataset_path = '/content/drive/MyDrive/demo_dataset_preprocessed/macenko'


file_type_counts = count_file_types(dataset_path)
print_file_type_counts(file_type_counts)



 folder 'MUSC_STROMA' file type:
  .tif: 904 files
  .png: 116 files

 folder 'ADI' file type:
  .png: 199 files
  .tif: 821 files

 folder 'LYM' file type:
  .tif: 922 files
  .png: 78 files

 folder 'NORM' file type:
  .tif: 801 files
  .png: 199 files

 folder 'DEBRIS_MUCUS' file type:
  .tif: 876 files
  .png: 124 files

 folder 'TUM' file type:
  .png: 182 files
  .tif: 818 files


###gaussian filter

In [9]:
import os
import numpy as np
import cv2
from PIL import Image

# Gaussian filter function
def apply_gaussian_filter(image):
    image_np = np.array(image)  # Convert image to NumPy array
    blurred_image = cv2.GaussianBlur(image_np, (5, 5), 0)  # Apply Gaussian filter
    return Image.fromarray(blurred_image)  # Convert back to PIL image

# Dataset paths
dataset_path = '/content/drive/MyDrive/demo_dataset'  # Input dataset path
save_base_path = '/content/drive/MyDrive/demo_dataset_preprocessed/gaussian'  # Output path for Gaussian-processed images

# Process all images in the dataset and apply Gaussian filter
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('.tif', '.png')):  # Only process .tif and .png images
            file_path = os.path.join(root, file)
            image = Image.open(file_path)

            # Apply Gaussian filter
            processed_image = apply_gaussian_filter(image)

            # Preserve original folder structure in the output folder
            relative_path = os.path.relpath(root, dataset_path)
            save_folder = os.path.join(save_base_path, relative_path)

            # Create the folder if it doesn't exist
            if not os.path.exists(save_folder):
                os.makedirs(save_folder)

            # Save the processed image in the original format
            save_extension = file.split('.')[-1].lower()
            save_path = os.path.join(save_folder, file)

            if save_extension == 'tif':
                processed_image.save(save_path, format='TIFF')
            elif save_extension == 'png':
                processed_image.save(save_path, format='PNG')

print("Gaussian filtering completed for all images.")


Gaussian filtering completed for all images.


In [10]:
import os
from collections import defaultdict

# file type
def count_file_types(dataset_path):
    file_type_counts = defaultdict(lambda: defaultdict(int))

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            file_extension = os.path.splitext(file)[1].lower()
            folder_name = os.path.basename(root)
            file_type_counts[folder_name][file_extension] += 1

    return file_type_counts


def print_file_type_counts(file_type_counts):
    for folder, types in file_type_counts.items():
        print(f"\n folder '{folder}' file type:")
        for file_type, count in types.items():
            print(f"  {file_type}: {count} files")


dataset_path = '/content/drive/MyDrive/demo_dataset_preprocessed/gaussian'


file_type_counts = count_file_types(dataset_path)
print_file_type_counts(file_type_counts)



 folder 'MUSC_STROMA' file type:
  .tif: 904 files
  .png: 116 files

 folder 'ADI' file type:
  .png: 199 files
  .tif: 821 files

 folder 'LYM' file type:
  .tif: 922 files
  .png: 78 files

 folder 'NORM' file type:
  .tif: 801 files
  .png: 199 files

 folder 'DEBRIS_MUCUS' file type:
  .tif: 876 files
  .png: 124 files

 folder 'TUM' file type:
  .png: 182 files
  .tif: 818 files


###rotation 0.2

In [11]:
import os
import random
from PIL import Image

# Rotation function: Rotate the image 90 degrees
def rotate_image(image):
    return image.rotate(90, expand=True)  # Rotate 90 degrees

# Dataset paths
dataset_path = '/content/drive/MyDrive/demo_dataset'  # Input dataset path
rotation_path = '/content/drive/MyDrive/demo_dataset_preprocessed/rotation'  # Output path for rotated images

# Collect all images in the dataset
all_images = []
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('.tif', '.png')):  # Only process .tif and .png images
            all_images.append(os.path.join(root, file))

print(f"Found {len(all_images)} images in the dataset.")

# Randomly select 20% of the images for rotation
rotation_images = random.sample(all_images, int(len(all_images) * 0.2))
rotation_images_set = set(rotation_images)  # Use a set for faster lookup
print(f"Selected {len(rotation_images)} images for rotation.")

# Process all images: rotate 20% and save all (rotated or original)
for file_path in all_images:
    image = Image.open(file_path)

    # Preserve original folder structure in the output folder
    relative_path = os.path.relpath(os.path.dirname(file_path), dataset_path)
    save_folder = os.path.join(rotation_path, relative_path)

    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # If the image is in the set to be rotated, apply rotation, otherwise keep the original
    if file_path in rotation_images_set:
        processed_image = rotate_image(image)
        print(f"Rotating and saving image: {file_path}")
    else:
        processed_image = image
        print(f"Saving original image (no rotation): {file_path}")

    # Save the processed image
    save_extension = file_path.split('.')[-1].lower()  # Get original extension
    save_path = os.path.join(save_folder, os.path.basename(file_path))

    if save_extension == 'tif':
        processed_image.save(save_path, format='TIFF')
    elif save_extension == 'png':
        processed_image.save(save_path, format='PNG')

print("Rotation processing completed for all images.")


流式输出内容被截断，只能显示最后 5000 行内容。
Saving original image (no rotation): /content/drive/MyDrive/demo_dataset/ADI/ADI-NEMDGEFH.tif
Saving original image (no rotation): /content/drive/MyDrive/demo_dataset/ADI/ADI-DIAYKKYN.tif
Saving original image (no rotation): /content/drive/MyDrive/demo_dataset/ADI/ADI-SGCAFLRY.tif
Saving original image (no rotation): /content/drive/MyDrive/demo_dataset/ADI/ADI-PDHMHYMM.tif
Saving original image (no rotation): /content/drive/MyDrive/demo_dataset/ADI/ADI-RGHEEPHL.tif
Saving original image (no rotation): /content/drive/MyDrive/demo_dataset/ADI/CRC_ADI_155 (1).png
Rotating and saving image: /content/drive/MyDrive/demo_dataset/ADI/ADI-EIVNEHVI.tif
Saving original image (no rotation): /content/drive/MyDrive/demo_dataset/ADI/ADI-NPYVRHTC.tif
Saving original image (no rotation): /content/drive/MyDrive/demo_dataset/ADI/CRC_ADI_137.png
Saving original image (no rotation): /content/drive/MyDrive/demo_dataset/ADI/ADI-GSYICRDL.tif
Saving original image (no rotation): /con

In [12]:
import os
from collections import defaultdict

# file type
def count_file_types(dataset_path):
    file_type_counts = defaultdict(lambda: defaultdict(int))

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            file_extension = os.path.splitext(file)[1].lower()
            folder_name = os.path.basename(root)
            file_type_counts[folder_name][file_extension] += 1

    return file_type_counts


def print_file_type_counts(file_type_counts):
    for folder, types in file_type_counts.items():
        print(f"\n folder '{folder}' file type:")
        for file_type, count in types.items():
            print(f"  {file_type}: {count} files")


dataset_path = '/content/drive/MyDrive/demo_dataset_preprocessed/rotation'


file_type_counts = count_file_types(dataset_path)
print_file_type_counts(file_type_counts)


 folder 'MUSC_STROMA' file type:
  .tif: 904 files
  .png: 116 files

 folder 'ADI' file type:
  .png: 199 files
  .tif: 821 files

 folder 'LYM' file type:
  .tif: 922 files
  .png: 78 files

 folder 'NORM' file type:
  .tif: 801 files
  .png: 199 files

 folder 'DEBRIS_MUCUS' file type:
  .tif: 876 files
  .png: 124 files

 folder 'TUM' file type:
  .png: 182 files
  .tif: 818 files


###macenko&Gaussian

In [16]:
import os
import numpy as np
from PIL import Image
from sklearn.decomposition import NMF
import cv2

# Apply Macenko normalization and Gaussian filter in combination
def apply_macenko_gaussian(image, reference_image):
    try:
        # Step 1: Macenko normalization
        image_np = np.array(image)
        ref_image_np = np.array(reference_image)

        height, width, _ = image_np.shape

        image_flat = image_np.reshape((-1, 3))
        ref_flat = ref_image_np.reshape((-1, 3))

        nmf_image = NMF(n_components=2)
        nmf_ref = NMF(n_components=2)

        W_image = nmf_image.fit_transform(image_flat)
        H_image = nmf_image.components_

        W_ref = nmf_ref.fit_transform(ref_flat)
        H_ref = nmf_ref.components_

        # Multiply the components to get the stains
        stains = np.dot(W_image, H_ref[:2, :])
        stains = stains.reshape((height, width, 3))
        stains = (stains - np.min(stains)) / (np.max(stains) - np.min(stains)) * 255
        stains_image = Image.fromarray(stains.astype(np.uint8))

        # Step 2: Gaussian filter
        blurred_image = cv2.GaussianBlur(np.array(stains_image), (5, 5), 0)
        return Image.fromarray(blurred_image)

    except Exception as e:
        print(f"Error during Macenko + Gaussian process: {e}")
        return None

# Dataset paths
dataset_path = '/content/drive/MyDrive/demo_dataset'
save_base_path = '/content/drive/MyDrive/demo_dataset_preprocessed/macenko_gaussian'

# Load reference image for Macenko normalization
reference_image_path = '/content/drive/MyDrive/target_image.tif'
reference_image = Image.open(reference_image_path)

# Process all images with both Macenko and Gaussian
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('.tif', '.png')):
            file_path = os.path.join(root, file)
            image = Image.open(file_path)

            # Apply Macenko + Gaussian
            processed_image = apply_macenko_gaussian(image, reference_image)

            if processed_image is not None:
                # Preserve folder structure
                relative_path = os.path.relpath(root, dataset_path)
                save_folder = os.path.join(save_base_path, relative_path)
                if not os.path.exists(save_folder):
                    os.makedirs(save_folder)

                # Save image
                save_extension = file.split('.')[-1].lower()
                save_path = os.path.join(save_folder, file)
                if save_extension == 'tif':
                    processed_image.save(save_path, format='TIFF')
                elif save_extension == 'png':
                    processed_image.save(save_path, format='PNG')

print("Macenko + Gaussian processing completed.")


Macenko + Gaussian processing completed.


In [17]:
import os
from collections import defaultdict

# file type
def count_file_types(dataset_path):
    file_type_counts = defaultdict(lambda: defaultdict(int))

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            file_extension = os.path.splitext(file)[1].lower()
            folder_name = os.path.basename(root)
            file_type_counts[folder_name][file_extension] += 1

    return file_type_counts


def print_file_type_counts(file_type_counts):
    for folder, types in file_type_counts.items():
        print(f"\n folder '{folder}' file type:")
        for file_type, count in types.items():
            print(f"  {file_type}: {count} files")


dataset_path = '/content/drive/MyDrive/demo_dataset_preprocessed/macenko_gaussian'


file_type_counts = count_file_types(dataset_path)
print_file_type_counts(file_type_counts)


 folder 'MUSC_STROMA' file type:
  .tif: 904 files
  .png: 116 files

 folder 'ADI' file type:
  .png: 199 files
  .tif: 821 files

 folder 'LYM' file type:
  .tif: 922 files
  .png: 78 files

 folder 'NORM' file type:
  .tif: 801 files
  .png: 199 files

 folder 'DEBRIS_MUCUS' file type:
  .tif: 876 files
  .png: 124 files

 folder 'TUM' file type:
  .png: 182 files
  .tif: 818 files


###Macenko + Rotation

In [6]:
import os
import random
from PIL import Image
import numpy as np
from sklearn.decomposition import NMF

# Apply Macenko normalization and randomly rotate 90 degrees
def apply_macenko_rotation(image, reference_image, rotate=False):
    try:
        # Macenko normalization
        image_np = np.array(image)
        ref_image_np = np.array(reference_image)
        height, width, _ = image_np.shape

        image_flat = image_np.reshape((-1, 3))
        ref_flat = ref_image_np.reshape((-1, 3))

        nmf_image = NMF(n_components=2)
        nmf_ref = NMF(n_components=2)

        W_image = nmf_image.fit_transform(image_flat)
        H_ref = nmf_ref.fit_transform(ref_flat)

        stains = np.dot(W_image, H_ref[:2, :]).reshape((height, width, 3))
        stains = (stains - np.min(stains)) / (np.max(stains) - np.min(stains)) * 255
        stains_image = Image.fromarray(stains.astype(np.uint8))

        # Apply 90-degree rotation if needed
        if rotate:
            stains_image = stains_image.rotate(90, expand=True)

        return stains_image
    except Exception as e:
        #print(f"Error during Macenko + Rotation process: {e}")
        return None

# Dataset paths
dataset_path = '/content/drive/MyDrive/demo_dataset'
save_base_path = '/content/drive/MyDrive/demo_dataset_preprocessed/macenko_rotation'

# Load reference image for Macenko normalization
reference_image_path = '/content/drive/MyDrive/target_image.tif'
reference_image = Image.open(reference_image_path)

# Collect all images and randomly select 20% for rotation
all_images = []
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('.tif', '.png')):
            all_images.append(os.path.join(root, file))

rotation_images = random.sample(all_images, int(len(all_images) * 0.2))
rotation_images_set = set(rotation_images)

# Process all images
for file_path in all_images:
    image = Image.open(file_path)
    rotate = file_path in rotation_images_set  # Rotate only 20% of the images

    # Apply Macenko + Rotation
    processed_image = apply_macenko_rotation(image, reference_image, rotate=rotate)

    if processed_image is not None:
        # Preserve folder structure
        relative_path = os.path.relpath(os.path.dirname(file_path), dataset_path)
        save_folder = os.path.join(save_base_path, relative_path)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)

        # Save image
        save_extension = file_path.split('.')[-1].lower()
        save_path = os.path.join(save_folder, os.path.basename(file_path))
        if save_extension == 'tif':
            processed_image.save(save_path, format='TIFF')
        elif save_extension == 'png':
            processed_image.save(save_path, format='PNG')

print("Macenko + Rotation processing completed.")


Macenko + Rotation processing completed.


###Gaussian + Rotation

In [7]:
import os
import random
import cv2
from PIL import Image

# Apply Gaussian filter and randomly rotate 90 degrees
def apply_gaussian_rotation(image, rotate=False):
    try:
        # Step 1: Gaussian filter
        image_np = np.array(image)
        blurred_image = cv2.GaussianBlur(image_np, (5, 5), 0)
        blurred_image_pil = Image.fromarray(blurred_image)

        # Step 2: Apply 90-degree rotation if needed
        if rotate:
            blurred_image_pil = blurred_image_pil.rotate(90, expand=True)

        return blurred_image_pil
    except Exception as e:
        print(f"Error during Gaussian + Rotation process: {e}")
        return None

# Dataset paths
dataset_path = '/content/drive/MyDrive/demo_dataset'
save_base_path = '/content/drive/MyDrive/demo_dataset_preprocessed/gaussian_rotation'

# Collect all images and randomly select 20% for rotation
all_images = []
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('.tif', '.png')):
            all_images.append(os.path.join(root, file))

rotation_images = random.sample(all_images, int(len(all_images) * 0.2))
rotation_images_set = set(rotation_images)

# Process all images
for file_path in all_images:
    image = Image.open(file_path)
    rotate = file_path in rotation_images_set  # Rotate only 20% of the images

    # Apply Gaussian + Rotation
    processed_image = apply_gaussian_rotation(image, rotate=rotate)

    if processed_image is not None:
        # Preserve folder structure
        relative_path = os.path.relpath(os.path.dirname(file_path), dataset_path)
        save_folder = os.path.join(save_base_path, relative_path)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)

        # Save image
        save_extension = file_path.split('.')[-1].lower()
        save_path = os.path.join(save_folder, os.path.basename(file_path))
        if save_extension == 'tif':
            processed_image.save(save_path, format='TIFF')
        elif save_extension == 'png':
            processed_image.save(save_path, format='PNG')

print("Gaussian + Rotation processing completed.")


Gaussian + Rotation processing completed.


In [8]:
import os
from collections import defaultdict

# file type
def count_file_types(dataset_path):
    file_type_counts = defaultdict(lambda: defaultdict(int))

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            file_extension = os.path.splitext(file)[1].lower()
            folder_name = os.path.basename(root)
            file_type_counts[folder_name][file_extension] += 1

    return file_type_counts


def print_file_type_counts(file_type_counts):
    for folder, types in file_type_counts.items():
        print(f"\n folder '{folder}' file type:")
        for file_type, count in types.items():
            print(f"  {file_type}: {count} files")


dataset_path = '/content/drive/MyDrive/demo_dataset_preprocessed/gaussian_rotation'


file_type_counts = count_file_types(dataset_path)
print_file_type_counts(file_type_counts)


 folder 'MUSC_STROMA' file type:
  .tif: 904 files
  .png: 116 files

 folder 'ADI' file type:
  .png: 199 files
  .tif: 821 files

 folder 'LYM' file type:
  .tif: 922 files
  .png: 78 files

 folder 'NORM' file type:
  .tif: 801 files
  .png: 199 files

 folder 'DEBRIS_MUCUS' file type:
  .tif: 876 files
  .png: 124 files

 folder 'TUM' file type:
  .png: 182 files
  .tif: 818 files


###Gaussian Macenko Rotation

In [3]:
import os
import random
import numpy as np
import cv2
from PIL import Image
from sklearn.decomposition import NMF

# Apply Macenko normalization, Gaussian filter, and randomly rotate 90 degrees
def apply_macenko_gaussian_rotation(image, reference_image, rotate=False):
    try:
        # Step 1: Macenko normalization
        image_np = np.array(image)
        ref_image_np = np.array(reference_image)
        height, width, _ = image_np.shape

        image_flat = image_np.reshape((-1, 3))
        ref_flat = ref_image_np.reshape((-1, 3))

        nmf_image = NMF(n_components=2)
        nmf_ref = NMF(n_components=2)

        W_image = nmf_image.fit_transform(image_flat)
        H_ref = nmf_ref.fit_transform(ref_flat)

        stains = np.dot(W_image, H_ref[:2, :]).reshape((height, width, 3))
        stains = (stains - np.min(stains)) / (np.max(stains) - np.min(stains)) * 255
        stains_image = Image.fromarray(stains.astype(np.uint8))

        # Step 2: Gaussian filter
        blurred_image = cv2.GaussianBlur(np.array(stains_image), (5, 5), 0)
        blurred_image_pil = Image.fromarray(blurred_image)

        # Step 3: Apply 90-degree rotation if needed
        if rotate:
            blurred_image_pil = blurred_image_pil.rotate(90, expand=True)

        return blurred_image_pil
    except Exception as e:
        #print(f"Error during Macenko + Gaussian + Rotation process: {e}")
        return None

# Dataset paths
dataset_path = '/content/drive/MyDrive/demo_dataset'
save_base_path = '/content/drive/MyDrive/demo_dataset_preprocessed/macenko_gaussian_rotation'

# Load reference image for Macenko normalization
reference_image_path = '/content/drive/MyDrive/target_image.tif'
reference_image = Image.open(reference_image_path)

# Collect all images and randomly select 20% for rotation
all_images = []
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('.tif', '.png')):
            all_images.append(os.path.join(root, file))

rotation_images = random.sample(all_images, int(len(all_images) * 0.2))
rotation_images_set = set(rotation_images)

# Process all images
for file_path in all_images:
    image = Image.open(file_path)
    rotate = file_path in rotation_images_set  # Rotate only 20% of the images

    # Apply Macenko + Gaussian + Rotation
    processed_image = apply_macenko_gaussian_rotation(image, reference_image, rotate=rotate)

    if processed_image is not None:
        # Preserve folder structure
        relative_path = os.path.relpath(os.path.dirname(file_path), dataset_path)
        save_folder = os.path.join(save_base_path, relative_path)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)

        # Save image
        save_extension = file_path.split('.')[-1].lower()
        save_path = os.path.join(save_folder, os.path.basename(file_path))
        if save_extension == 'tif':
            processed_image.save(save_path, format='TIFF')
        elif save_extension == 'png':
            processed_image.save(save_path, format='PNG')

print("Macenko + Gaussian + Rotation processing completed.")




Macenko + Gaussian + Rotation processing completed.
